In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
from llama_index.core import SimpleDirectoryReader,VectorStoreIndex,SummaryIndex,Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core.tools import QueryEngineTool
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector
from llama_index.llms.openai import OpenAI

/Users/alina.ghani/VS_project/hotel_data/ContentGen/ENV/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 1024

print("creating instance of LLM")
Settings.llm = OpenAI(model="gpt-3.5-turbo")

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 59409.41it/s]


creating instance of LLM


In [5]:

documents = SimpleDirectoryReader(input_files = ['/Users/alina.ghani/VS_project/chatbot_dan/data/doc/V11_Argumentaire_Peugeot_2024.pdf']).load_data()
print(len(documents))
print(f"Document Metadata: {documents[0].metadata}")

splitter = SentenceSplitter(chunk_size=1024,chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
print(f"Length of nodes : {len(nodes)}")

/Users/alina.ghani/VS_project/hotel_data/ContentGen/ENV/lib/python3.12/site-packages/pypdf/_crypt_providers/_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


44
Document Metadata: {'page_label': '1', 'file_name': 'V11_Argumentaire_Peugeot_2024.pdf', 'file_path': '/Users/alina.ghani/VS_project/chatbot_dan/data/doc/V11_Argumentaire_Peugeot_2024.pdf', 'file_type': 'application/pdf', 'file_size': 12583420, 'creation_date': '2024-07-30', 'last_modified_date': '2024-07-21'}
Length of nodes : 44


In [6]:

summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)
vector_query_engine = vector_index.as_query_engine()


summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "Useful for summarization questions related to the document"
    ),
)

vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "Useful for retrieving specific context from the document."
    ),
)

query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

response = query_engine.query("Explain the document i want a summury  ?")
print(str(response))
print(len(response.source_nodes))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Selecting query engine 0: Useful for summarization questions related to the document.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


The document provides a comprehensive overview of Peugeot's electric vehicle lineup, detailing various models like E-208, E-2008, E-3008, E-308, E-308 SW, 508, E-Rifter, E-Traveller, E-Partner, E-Expert, and E-Boxer. It covers key aspects such as battery technology, charging capabilities, range, emissions, and charging times for each model. Additionally, it emphasizes the importance of connected mobility services like Connect One, Connect Plus, and Connect Premium packages, along with applications such as MyPeugeot, E-Routes, and eSolutions Charging for efficient monitoring and management of electric vehicles. The document also discusses government incentives, fiscal policies, extended warranties, and maintenance programs that highlight Peugeot's commitment to quality and reliability in the electric vehicle market.
44


# RouterQueryEngine for summarization and context retrieval.

In [3]:
import os
from dotenv import load_dotenv
from elasticsearch import Elasticsearch

# Load environment variables
load_dotenv()

# Elasticsearch configurations with increased timeouts
es_client = Elasticsearch(
    "https://371e52c2ddc94eeda8d2dbeb8acc5645.us-central1.gcp.cloud.es.io:443",
    api_key=os.environ["elastic_host"],
    verify_certs=False,
    request_timeout=120,
    max_retries=10,
    retry_on_timeout=True
)


## key
index_source_fields = {
    "peugeot_ev_pdf": ["content"],
}


/Users/alina.ghani/VS_project/hotel_data/ContentGen/ENV/lib/python3.12/site-packages/elasticsearch/_sync/client/__init__.py:399: SecurityWarning: Connecting to 'https://371e52c2ddc94eeda8d2dbeb8acc5645.us-central1.gcp.cloud.es.io:443' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [4]:
from llama_index.core import Document

def retrieve_documents(query):
    response = es_client.search(
        index="peugeot_ev_pdf",
        body={
            "query": {
                "match": {
                    "content": query
                }
            }
        }
    )
    hits = response['hits']['hits']
    documents = []
    for hit in hits:
        metadata = hit['_source']['metadata']
        # Trim down metadata and handle missing keys
        essential_metadata = {key: metadata.get(key, 'N/A') for key in ('title', 'author', 'date')}
        documents.append(Document(text=hit['_source']['content'], extra_info=essential_metadata))
    return documents

# Example query
query = "Explain the document, I want a summary."
documents = retrieve_documents(query)

# Initialize SentenceSplitter with a larger chunk size to handle larger metadata
splitter = SentenceSplitter(chunk_size=20000, chunk_overlap=100)
nodes = splitter.get_nodes_from_documents(documents)
nodes = splitter.get_nodes_from_documents(documents)

/Users/alina.ghani/VS_project/hotel_data/ContentGen/ENV/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '371e52c2ddc94eeda8d2dbeb8acc5645.us-central1.gcp.cloud.es.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


NameError: name 'Document' is not defined

In [45]:
# Set up the embedding model and language model
embed_model = FastEmbedEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embed_model
Settings.chunk_size = 1024

print("Creating instance of LLM")
Settings.llm = OpenAI(model="gpt-3.5-turbo")

Fetching 5 files: 100%|██████████| 5/5 [00:00<00:00, 33770.56it/s]

Creating instance of LLM


In [47]:
# Create indices
summary_index = SummaryIndex(nodes)
vector_index = VectorStoreIndex(nodes)

# Create query engines
summary_query_engine = summary_index.as_query_engine(response_mode="tree_summarize", use_async=True)
vector_query_engine = vector_index.as_query_engine()

# Create query tools
summary_tool = QueryEngineTool.from_defaults(query_engine=summary_query_engine, description="Useful for summarization questions related to the document")
vector_tool = QueryEngineTool.from_defaults(query_engine=vector_query_engine, description="Useful for retrieving specific context from the document.")

# Create router query engine
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[summary_tool, vector_tool],
    verbose=True
)

# Perform a query
response = query_engine.query(query)
print(str(response))

Selecting query engine 0: Useful for summarization questions related to the document.
The document provides detailed information on electric vehicles, including incentives and regulations in France. It covers topics such as the environmental impact score, fiscal benefits, types of electric vehicles available, charging infrastructure, and power requirements for residential electric installations. Additionally, it discusses the criteria for receiving eco-bonuses, the importance of the environmental score, and the different types of electric vehicle technologies available in the market.


In [48]:
query2 = "talk me about e-208"
response2 = query_engine.query(query2)
print(str(response2))


Selecting query engine 1: Useful for retrieving specific context from the document..
The e-208 is one of the electric vehicle models eligible for the Wallbox offer mentioned in the text. It is available for both individual customers and businesses. Additionally, the e-208 is part of the vehicles that qualify for the prime à la conversion, which provides incentives for purchasing 100% electric vehicles.
